### TODO
- tc model for two variables with no correlated erros - use state space inference library
- estimate on simulated data
- estimate on real data
- add var error correlation
- add trend correlation
- extedn to more variables

In [66]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [67]:
using Revise
using LinearAlgebra
using Distributions
using Plots
using StateSpaceInference

includet("TCVAR_model.jl")

In [71]:
params_true = NamedTuple{(:στπ, :στr, :Φ11, :Φ12, :Φ21, :Φ22, :σcπ, :σcr)}((.02, .01, .6, .3, .4, .2, .015, .01))
   

TCVAR(params_true, [.02, .01], diagm([.01, .005]))



MethodError: MethodError: no method matching LinearGaussianStateSpaceModel(::Vector{Float64}, ::Matrix{Float64}, ::Matrix{Float64}, ::Vector{Float64}, ::Matrix{Float64}, ::Matrix{Int64}, ::Matrix{Float64})
The type `LinearGaussianStateSpaceModel` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  LinearGaussianStateSpaceModel(::Vector{XT}, ::AbstractMatrix{XT}, ::Matrix{XT}, ::Vector{XT}, ::AbstractMatrix{XT}, !Matched::Matrix{XT}, ::AbstractMatrix{XT}) where XT
   @ StateSpaceInference C:\Users\matsz\.julia\packages\StateSpaceInference\EJ9sI\src\linear_models.jl:53
  LinearGaussianStateSpaceModel(::Vector{XT}, !Matched::PDMats.PDMat{XT, S} where S<:AbstractMatrix{XT}, ::Matrix{XT}, ::Vector{XT}, !Matched::PDMats.PDMat{XT, S} where S<:AbstractMatrix{XT}, !Matched::Matrix{XT}, !Matched::PDMats.PDMat{XT, S} where S<:AbstractMatrix{XT}) where XT<:Number
   @ StateSpaceInference C:\Users\matsz\.julia\packages\Stat

In [72]:
Diagonal([1,2,3])

3×3 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅
 ⋅  2  ⋅
 ⋅  ⋅  3